In [5]:
import json
import os

In [6]:
path = "/Users/jessicabuzzelli/repos/bdh-reproducibility-challenge/Survival-MDN/output/"

In [28]:
# save improperly formatted json as txt
folder_names = ['gbsg', 'metabric', 'support']
model_ids = list(range(100))

for folder_name in folder_names:
    for subfolder_name in [f"split{i}" for i in range(1, 11)]:
        for model_id in model_ids:
            json_name = f'model_{model_id}_eval.json'
            fixed_name = f'model_{model_id}_eval.txt'
            
            os.rename(os.path.join(path, folder_name, subfolder_name, json_name), os.path.join(path, folder_name, subfolder_name, fixed_name))

In [7]:
filename = "gbsg/split1/model_0_eval.txt"

with open(path + filename, 'rb') as f:
    eval_data = json.loads(f.read().decode("utf-16").replace("'", '"'))

In [8]:
eval_data

{'valid': [{'survival_loss': [0.36488041281700134],
   'ipw_concordance_time_dependent': [0.6746823175342562],
   'ipw_2_concordance_time_dependent': [0.6746823175342562],
   'ipw_4_concordance_time_dependent': [0.6746823175342562],
   'brier_score': 0.15433688518787994,
   'brier_score_2': 0.15435662594943378,
   'brier_score_4': 0.15435662594943378,
   'binomial_log_likelihood': -0.4619125662597353,
   'binomial_log_likelihood_2': -0.4619551460465026,
   'binomial_log_likelihood_4': -0.4619551460465026}],
 'test': [{'survival_loss': [0.3593335449695587],
   'ipw_concordance_time_dependent': [0.6398132839036762],
   'ipw_2_concordance_time_dependent': [0.6398132839036762],
   'ipw_4_concordance_time_dependent': [0.6398132839036762],
   'brier_score': 0.18028053699028165,
   'brier_score_2': 0.18044388688561003,
   'brier_score_4': 0.18044388688561003,
   'binomial_log_likelihood': -0.5310803220204467,
   'binomial_log_likelihood_2': -0.531483218279963,
   'binomial_log_likelihood_4': 

In [15]:
folder_names = ['gbsg', 'metabric', 'support']
model_ids = list(range(100))

eval_metrics = {f: [] for f in folder_names}
for folder_name in folder_names:
    for subfolder_name in [f"split{i}" for i in range(1, 11)]:
        for model_id in model_ids:
            iteration = {}
            eval_file_name = f'model_{model_id}_eval.txt'
            
            # Load the evaluation metrics
            print()
            try:
                with open(os.path.join(path, folder_name, subfolder_name, eval_file_name), 'rb') as f:
                    eval_data = json.loads(f.read().decode("utf-16").replace("'", '"'))['valid'][0]
            
                iteration['loss'] = eval_data['survival_loss'][0]
                iteration['concordance'] = eval_data['ipw_concordance_time_dependent'][0]
                iteration['brier'] = eval_data['brier_score']
                iteration['binomial_log_likelihood'] = eval_data['binomial_log_likelihood']
            
                eval_metrics[folder_name].append(iteration)
            except:
                pass

In [30]:
from statistics import stdev, mean, median

In [32]:
for dataset in ['gbsg', 'metabric', 'support']:
    loss, concordance, brier, binll = [],[],[],[]
    for s in range(1, 11):
        for n in range(100):
            eval_file_name = f"/Users/jessicabuzzelli/repos/bdh-reproducibility-challenge/Survival-MDN/output/gbsg/split{s}/model_{n}_eval.txt"
            
            try:
                with open(os.path.join(path, folder_name, subfolder_name, eval_file_name), 'rb') as f:
                    eval_data = json.loads(f.read().decode("utf-16").replace("'", '"'))['valid'][0]
                            
                    loss.append(eval_data['survival_loss'][0])
                    concordance.append(eval_data['ipw_concordance_time_dependent'][0])
                    brier.append(eval_data['brier_score'])
                    binll.append(eval_data['binomial_log_likelihood'])
            except:
                pass
    
    with open("results.txt", "a") as f:
        f.write(f"Dataset: {dataset}\n")

        for metric, data in zip(['loss', 'concordance', 'brier', 'binll'], [loss, concordance, brier, binll]):
            f.write(f"\tmetric: {metric}\n")
            f.write(f"\t\tmean: {mean(data)}\n")
            f.write(f"\t\tmedian: {median(data)}\n")
            f.write(f"\t\tstddev: {stdev(data)}\n")
            